In [1]:
# from os import chdir
# from google.colab import drive

# drive.mount("/content/drive", force_remount=True)
# chdir("/content/drive/MyDrive/Eliott/files/")

In [13]:
# IMPORTS

import numpy as np
import pandas as pd
import preprocessing as pp



In [14]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'

X_train, y_train = pp.prepare_dataframe(file_name_train)
X_test, y_test = pp.prepare_dataframe(file_name_test)

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

# Recherche d'Hyperparamètres

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

### SGD

In [16]:


grid_params = {
  'clf__penalty': ['l2', 'l1', 'elasticnet'],
  'clf__alpha': np.linspace(1e-7, 1e-4, 20),  
}



pipeline_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])


sgd_clf = GridSearchCV(pipeline_clf, grid_params,verbose=1, scoring='accuracy' ,n_jobs=-1)








In [ ]:
%%time
sgd_best = sgd_clf.fit(X_train, y_train)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [173]:
print("Best Score: ", sgd_clf.best_score_)
print("Best Params: ", sgd_clf.best_params_)



# Feed the training data through the pipeline
#text_clf.fit(X_train, y_train) 


Best Score:  0.8789757345568008
Best Params:  {'clf__alpha': 1.5873684210526315e-05, 'clf__penalty': 'l1'}


### Gradient boosting

In [176]:
from sklearn.ensemble import GradientBoostingClassifier

In [177]:
GB_param = {
    "gb_clf__learning_rate": [0.01, 0.1, 0.2],
    "gb_clf__n_estimators":[10,50,100,200,400,800]
    }

In [178]:
pipeline_gradient_boosting = Pipeline([('tfidf', TfidfVectorizer()),
                     ('gb_clf', GradientBoostingClassifier()),
])


gb_clf = GridSearchCV(pipeline_gradient_boosting, GB_param, scoring='accuracy', verbose=1 ,n_jobs=-1)

In [179]:
%%time
subset = X_train.shape[0]

gb_best = gb_clf.fit(X_train[:subset], y_train[:subset])    

Fitting 5 folds for each of 18 candidates, totalling 90 fits
CPU times: user 9min 29s, sys: 258 ms, total: 9min 29s
Wall time: 46min 55s


In [180]:
print("Best Score: ", gb_clf.best_score_)
print("Best Params: ", gb_clf.best_params_)

Best Score:  0.8194232669897714
Best Params:  {'gb_clf__learning_rate': 0.2, 'gb_clf__n_estimators': 800}


### Random Forest

In [181]:
randForest_param = {
    'rf_clf__n_estimators': [10, 100],   
    'rf_clf__max_depth': [10 ,150,300,600],
    'rf_clf__min_samples_leaf': [1, 2, 3],   
    'rf_clf__min_samples_split': [4, 8, 16, 32],
    'rf_clf__max_features': ['log2', 'sqrt'],
    'rf_clf__criterion': ['gini', 'entropy'],
    'rf_clf__warm_start': [True, False] 
}



pipeline_random_forest = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rf_clf', RandomForestClassifier()),
])


rf_clf = GridSearchCV(pipeline_random_forest, randForest_param, scoring='accuracy', verbose=1 ,n_jobs=-1)


In [182]:
%%time
subset = X_train.shape[0]

rf_best = rf_clf.fit(X_train[:subset], y_train[:subset])



Fitting 5 folds for each of 768 candidates, totalling 3840 fits
CPU times: user 1min 34s, sys: 8.71 s, total: 1min 43s
Wall time: 42min 30s


In [183]:
print("Best Score: ", rf_clf.best_score_)
print("Best Params: ", rf_clf.best_params_)

Best Score:  0.6925428813940141
Best Params:  {'rf_clf__criterion': 'entropy', 'rf_clf__max_depth': 300, 'rf_clf__max_features': 'sqrt', 'rf_clf__min_samples_leaf': 2, 'rf_clf__min_samples_split': 32, 'rf_clf__n_estimators': 100, 'rf_clf__warm_start': True}


### Logistic Regression

In [184]:
LogisticRegression_param = {
    'lr_clf__C': [100, 80, 40,20,10, 1.0],
    'lr_clf__tol': np.linspace(1e-8,1e-4,15)
}


In [185]:
pipeline_lr = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('lr_clf', LogisticRegression(max_iter=1000)),
 ])

In [186]:
lr_clf = GridSearchCV(pipeline_lr, LogisticRegression_param,  scoring='accuracy', verbose=1, n_jobs=-1)

In [187]:
%%time
lr_best = lr_clf.fit(X_train[:subset], y_train[:subset])

Fitting 5 folds for each of 90 candidates, totalling 450 fits
CPU times: user 1min 55s, sys: 4min 27s, total: 6min 23s
Wall time: 32min 56s


In [188]:
print("Best Score: ", lr_clf.best_score_)
print("Best Params: ", lr_clf.best_params_)

Best Score:  0.8230920602963577
Best Params:  {'lr_clf__C': 10, 'lr_clf__tol': 1e-08}


### Perceptron

In [64]:
from sklearn.linear_model import Perceptron


In [189]:
per_params = {
  'per_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'per_clf__alpha': np.linspace(1e-8, 1e-4, 20),
}

In [190]:
pipeline_perceptron = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('per_clf', Perceptron()),
 ])

In [191]:
per_clf = GridSearchCV(pipeline_perceptron, per_params,  scoring='accuracy', verbose=1, n_jobs=-1)

In [192]:
%%time
per_best = per_clf.fit(X_train[:subset], y_train[:subset])

Fitting 5 folds for each of 60 candidates, totalling 300 fits
CPU times: user 7.8 s, sys: 3.47 s, total: 11.3 s
Wall time: 41.6 s


In [193]:
print("Best Score: ", per_clf.best_score_)
print("Best Params: ", per_clf.best_params_)

Best Score:  0.8128628567555942
Best Params:  {'per_clf__alpha': 5.272631578947369e-06, 'per_clf__penalty': 'l1'}


#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [70]:
from sklearn.svm import LinearSVC


In [71]:
svc_params = {
  'svc_clf__penalty': ['l2', 'l1', 'elasticnet'],
  'svc_clf__loss': ['hinge', 'squared_hinge'],
  'svc_clf__dual' : [False,True]
}

In [72]:
pipeline_svc = Pipeline([('tfidf', TfidfVectorizer()),
                       
                     ('svc_clf', LinearSVC(max_iter=10000)),
 ])

In [73]:
svc_clf = GridSearchCV(pipeline_svc, svc_params,  scoring='accuracy', verbose=1, n_jobs=-1)

In [78]:
%%time
svc_best = svc_clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/eliott/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/eliott/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, self.intercept_, self.n_it

CPU times: user 6.8 s, sys: 151 ms, total: 6.95 s
Wall time: 27 s


In [81]:
print("Best Score: ", svc_clf.best_score_)
print("Best Params: ", svc_clf.best_params_)

Best Score:  0.8689651633428769
Best Params:  {'svc_clf__dual': False, 'svc_clf__loss': 'squared_hinge', 'svc_clf__penalty': 'l1'}


### Saving Hyperparameters in JSON

In [82]:
import json

In [194]:
dict = {}
dict['SGD'] = sgd_clf.best_params_
dict['RF'] = rf_clf.best_params_
dict['GB'] = gb_clf.best_params_
dict['LR'] = lr_clf.best_params_
dict['Perceptron'] = per_clf.best_params_
dict['SVC'] = svc_clf.best_params_


with open('data.json', 'w') as fp:
    json.dump(dict, fp,  indent=4)

### Charging the best parameters for training

In [197]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)


data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']



### Training on all training data and testing on test data

In [198]:
from sklearn.metrics import accuracy_score, f1_score

In [196]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  ")




### SGD

In [199]:
model_sgd = pipeline_clf
model_sgd.set_params(**data_sgd) # clf__max_iter=1000

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.5873684210526315e-05, penalty='l1'))])

In [200]:
%%time

model_sgd.fit(X_train,y_train)


CPU times: user 1.69 s, sys: 2.77 s, total: 4.46 s
Wall time: 1.23 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.5873684210526315e-05, penalty='l1'))])

In [201]:
testing(model_sgd)

train_accuracy : 0.923415214908764 
test_accuracy : 0.882306477093207  


In [ ]:
f1score = f1_score(y_test, ad_best_ss.predict(X_test_ss), average='weighted')


### Random Forest

In [202]:
model_rf = pipeline_random_forest
model_rf.set_params(**data_rf) # clf__max_iter=1000

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('rf_clf',
                 RandomForestClassifier(criterion='entropy', max_depth=300,
                                        max_features='sqrt', min_samples_leaf=2,
                                        min_samples_split=32,
                                        warm_start=True))])

In [203]:
%%time

model_rf.fit(X_train,y_train)

CPU times: user 10.2 s, sys: 628 µs, total: 10.2 s
Wall time: 10.2 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('rf_clf',
                 RandomForestClassifier(criterion='entropy', max_depth=300,
                                        max_features='sqrt', min_samples_leaf=2,
                                        min_samples_split=32,
                                        warm_start=True))])

In [204]:
testing(model_rf)

train_accuracy : 0.8715163884636878 
test_accuracy : 0.6951026856240127  


### Gradient Boosting 

In [205]:
model_gb = pipeline_gradient_boosting
model_gb.set_params(**data_gb)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('gb_clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [206]:
%%time

model_gb.fit(X_train,y_train)

CPU times: user 9min 31s, sys: 5.85 ms, total: 9min 31s
Wall time: 9min 32s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('gb_clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [207]:
testing(model_gb)

train_accuracy : 0.9167334839759944 
test_accuracy : 0.8299104791995787  


### Logistic Regression

In [208]:
model_lr = pipeline_lr
model_lr.set_params(**data_lr)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr_clf', LogisticRegression(C=10, max_iter=1000, tol=1e-08))])

In [209]:
%%time

model_lr.fit(X_train,y_train)

CPU times: user 1min 16s, sys: 3min 54s, total: 5min 11s
Wall time: 22.6 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr_clf', LogisticRegression(C=10, max_iter=1000, tol=1e-08))])

In [210]:
testing(model_lr)

train_accuracy : 0.9755327161843672 
test_accuracy : 0.833070036861506  


### Perceptron

In [211]:
model_per = pipeline_perceptron
model_per.set_params(**data_per)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('per_clf',
                 Perceptron(alpha=5.272631578947369e-06, penalty='l1'))])

In [212]:
%%time

model_per.fit(X_train,y_train)

CPU times: user 1.52 s, sys: 2.73 s, total: 4.26 s
Wall time: 1.07 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('per_clf',
                 Perceptron(alpha=5.272631578947369e-06, penalty='l1'))])

In [213]:
testing(model_per)

train_accuracy : 0.9055324732123332 
test_accuracy : 0.822538177988415  


### SVC

In [214]:
model_svc = pipeline_svc
model_svc.set_params(**data_svc)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('svc_clf',
                 LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [215]:
%%time

model_svc.fit(X_train,y_train)

CPU times: user 5.51 s, sys: 6.3 ms, total: 5.52 s
Wall time: 5.52 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('svc_clf',
                 LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [216]:
testing(model_svc)

train_accuracy : 0.9481983623684914 
test_accuracy : 0.8730911005792522  


### Voting Classifier

In [163]:
from sklearn.ensemble import VotingClassifier


In [217]:
clf1 = model_sgd

clf2 = model_rf

clf3 = model_gb

clf4 = model_lr

clf5 = model_per

clf6 = model_svc


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


In [218]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 11min 2s, sys: 3min 55s, total: 14min 57s
Wall time: 10min 6s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.5873684210526315e-05,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('rf_clf',
                                               RandomForestClassifier(criterion='entropy',
                                                                      max_depth=300,
                                                                      max_features='sqrt',
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=32,
                                            

In [219]:
testing(eclf1)

train_accuracy : 0.9558276842335447 
test_accuracy : 0.8699315429173249  


In [220]:
%%time

eclf2.fit(X_train,y_train)

CPU times: user 1min 34s, sys: 4min 33s, total: 6min 8s
Wall time: 33.1 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.5873684210526315e-05,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('lr_clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                                  tol=1e-08))])),
                             ('per',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('per_clf',
                                               Perceptron(alpha=5.272631578947369e-06,


In [221]:
testing(eclf2)

train_accuracy : 0.9555361177928421 
test_accuracy : 0.8715113217482886  


In [237]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [238]:
predictions_out = eclf2.predict(tests_perso)


In [239]:
predictions_out

array([-1,  1,  1])